In [1]:
# -- coding: utf-8 -

In [2]:
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import dialogflow
import os
import json

import requests

In [ ]:
with open('data.txt', 'r') as f:
    api_keys = json.load(f)

In [2]:
updater = Updater(token=api_keys["telegramm_api"]) # Токен API к Telegram
dispatcher = updater.dispatcher

<ipython-input-2-d330b666dc0c>:1: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  updater = Updater(token='1235931628:AAFm1EYdDbC27skeu4D0smoT8oXZEdGdh08') # Токен API к Telegram


In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'gb-nlp-vv-0310-xqwh-d9d05c2c5bbd.json'# скачнный JSON

DIALOGFLOW_PROJECT_ID = 'gb-nlp-vv-0310-xqwh' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'GB_NLP_VV_0310_bot'  # ID бота из телеграма

In [4]:
def getWeatherByCity(city):
    url = "https://community-open-weather-map.p.rapidapi.com/weather"

    querystring = {"units":"metric",
                   "lang":"ru",
                   "q":city}

    headers = {
        'x-rapidapi-host': "community-open-weather-map.p.rapidapi.com",
        'x-rapidapi-key': api_keys["open_weather_api"]
        }

    response = requests.request("GET", url, headers=headers, params=querystring)    
    response_json = response.json()
    
    result_text = f"Погода в городе {city}: температура {response_json['main']['temp']}"
    weather = response_json["weather"]
    for w in weather:
        result_text += f" и {w['description']}"
    return result_text

In [5]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Добрый день')
    
def textMessage(bot, update):
    
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
    
    text_input = dialogflow.types.TextInput(text=update.message.text, language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.types.QueryInput(text=text_input)
    try:
        response = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
         raise
    
    if response.query_result.intent.display_name == "Intent_Weather":
        city = response.query_result.parameters["location"]["city"]
        text = getWeatherByCity(city)
        bot.send_message(chat_id=update.message.chat_id, text=text)
    else:            
        text = response.query_result.fulfillment_text
        if text:
            bot.send_message(chat_id=update.message.chat_id, text=text)
        else:
            bot.send_message(chat_id=update.message.chat_id, text='что?')

In [ ]:
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)

dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)

updater.start_polling(clean=True)
updater.idle()

{'coord': {'lon': 37.62, 'lat': 55.75}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'пасмурно', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 17.67, 'feels_like': 13.56, 'temp_min': 17, 'temp_max': 18, 'pressure': 1029, 'humidity': 51}, 'visibility': 10000, 'wind': {'speed': 5, 'deg': 130}, 'clouds': {'all': 96}, 'dt': 1601720588, 'sys': {'type': 1, 'id': 9029, 'country': 'RU', 'sunrise': 1601696195, 'sunset': 1601737210}, 'timezone': 10800, 'id': 524901, 'name': 'Москва', 'cod': 200}
